### Loading & preparing edgelist

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib
import os
from time import sleep
from tqdm import tqdm
ROOTPATH = os.getcwd()[:-3]

### Extracting triads with at least one edge reciprocated

In [2]:
wikipedia = pd.read_csv(ROOTPATH + "data/wikipedia_triads_sequential_edgelist/wikipedia_triads_sequential_edgelist.csv")
wikipedia = wikipedia[~wikipedia['fourth_link_node_1'].isnull()]
wikipedia.head(3)

,Unnamed: 0,node_1,node_2,node_3,first_link_node_1,first_link_node_2,first_link_sign,second_link_node_1,second_link_node_2,second_link_sign,...,fourth_link_node_1,fourth_link_node_2,fourth_link_sign,fifth_link_node_1,fifth_link_node_2,fifth_link_sign,sixth_link_node_1,sixth_link_node_2,sixth_link_sign,triad_context
24,24,2349.0,6086.0,4911.0,2349.0,4911.0,1.0,2349.0,6086.0,1.0,...,4911.0,2349.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,t9
25,25,6025.0,2349.0,6245.0,6025.0,2349.0,-1.0,2349.0,6245.0,-1.0,...,2349.0,6025.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,t16
27,27,1152.0,1234.0,1062.0,1234.0,1152.0,1.0,1062.0,1152.0,1.0,...,1152.0,1062.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,t3


In [3]:
Column_name1 = ['first_link_node_1','second_link_node_1','third_link_node_1','fourth_link_node_1','fifth_link_node_1','sixth_link_node_1']
Column_name2 = ['first_link_node_2','second_link_node_2','third_link_node_2','fourth_link_node_2','fifth_link_node_2','sixth_link_node_2']

link_name = ['first','second','third']

unbalanced = 0
balanced = 0

bp=0
up=0

bpp=0
bnn=0
upp=0
unn=0

#Surprisingly(?), no edge is reciprocated before the triad is closed. This seems very strange. 

for n in tqdm(range(len(wikipedia))):
    triad_set = []
    count = 0
    for n1,n2 in zip(Column_name1,Column_name2):
        count+=1
        if((wikipedia[n2].iloc[n],wikipedia[n1].iloc[n]) not in triad_set):
            triad_set.append((wikipedia[n1].iloc[n],wikipedia[n2].iloc[n]))
            if(len(triad_set)>2):
                break
   
    sign_prod = (wikipedia['first_link_sign'].iloc[n])*(wikipedia['second_link_sign'].iloc[n])*(wikipedia['third_link_sign'].iloc[n])
    
    if sign_prod>0:
        balanced+=1
        ind = triad_set.index((wikipedia['fourth_link_node_2'].iloc[n],wikipedia['fourth_link_node_1'].iloc[n]))
        if(wikipedia[link_name[ind]+'_link_sign'].iloc[n]>0):
            bp+=1
            if(wikipedia['fourth_link_sign'].iloc[n]>0):
                bpp+=1
        elif(wikipedia[link_name[ind]+'_link_sign'].iloc[n]<0):
            if(wikipedia['fourth_link_sign'].iloc[n]<0):
                bnn+=1
    elif sign_prod<0:
        unbalanced+=1
        ind = triad_set.index((wikipedia['fourth_link_node_2'].iloc[n],wikipedia['fourth_link_node_1'].iloc[n]))
        if(wikipedia[link_name[ind]+'_link_sign'].iloc[n]>0):
            up+=1
            if(wikipedia['fourth_link_sign'].iloc[n]>0):
                upp+=1
        elif(wikipedia[link_name[ind]+'_link_sign'].iloc[n]<0):
            if(wikipedia['fourth_link_sign'].iloc[n]<0):
                unn+=1
                
print('Balanced-->')
print('Triads:',round(balanced,3))
print('P(RSS):',round((bpp+bnn)/balanced,3))
print('P(+|+):',round(bpp/bp,3))
print('P(-|-):',round(bnn/(balanced-bp),3))

print('Unbalanced-->')
print('Triads:',round(unbalanced,3))
print('P(RSS):',round((upp+unn)/unbalanced,3))
print('P(+|+):',round(upp/up,3))
print('P(-|-):',round(unn/(unbalanced-up),3))

100%|███████████████████████████████████| 68077/68077 [00:10<00:00, 6258.20it/s]

Balanced-->
Triads: 54374
P(RSS): 0.912
P(+|+): 0.935
P(-|-): 0.331
Unbalanced-->
Triads: 13703
P(RSS): 0.658
P(+|+): 0.877
P(-|-): 0.193
